<a href="https://colab.research.google.com/github/MOFU0712/it_news_check_and_analysis/blob/main/it_news_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install japanize_matplotlib

# ワードクラウド生成用
!pip3 install wordcloud

# 日本語テキスト処理用
!pip3 install mecab-python3 unidic-lite

# 自然言語処理用
!pip3 install nltk

# ネットワークグラフ可視化用
!pip3 install networkx

# Notionとの接続
!pip3 install notion_client

# フォントのインストール
!apt-get -qq update
!apt-get -qq install -y fonts-ipafont-gothic fonts-ipafont-mincho

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
from collections import Counter
import re
from wordcloud import WordCloud
import matplotlib.font_manager as fm
from matplotlib.font_manager import FontProperties
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import MeCab
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import japanize_matplotlib
from google.colab import userdata


# 日本語フォントプロパティの設定
font_prop = FontProperties(fname=font_path)

# Colabのシークレットから認証情報を取得
NOTION_API_KEY = userdata.get('NOTION_API_KEY')
DATABASE_ID = userdata.get('PICKUP_DATABASE_KEY')

HEADERS = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def get_database_schema():
    """データベースのスキーマを取得する"""
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}"
    response = requests.get(url, headers=HEADERS)
    return response.json()

def query_database(start_date=None, end_date=None):
    """指定された期間のデータベースエントリを取得する"""
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"

    # フィルタの設定
    filter_params = {}
    if start_date and end_date:
        filter_params = {
            "and": [
                {
                    "property": "Created time",
                    "date": {
                        "on_or_after": start_date
                    }
                },
                {
                    "property": "Created time",
                    "date": {
                        "on_or_before": end_date
                    }
                }
            ]
        }

    # ページネーションのための変数
    has_more = True
    next_cursor = None
    results = []

    # ページネーションで全結果を取得
    while has_more:
        body = {"page_size": 100}
        if filter_params:
            body["filter"] = filter_params
        if next_cursor:
            body["start_cursor"] = next_cursor

        response = requests.post(url, headers=HEADERS, json=body)
        data = response.json()

        results.extend(data.get("results", []))
        has_more = data.get("has_more", False)
        next_cursor = data.get("next_cursor")

    return results

def extract_properties(results):
    """Notionの結果からプロパティを抽出してDataFrameに変換する"""
    articles = []

    for result in results:
        properties = result.get("properties", {})
        article = {}

        # タイトルを取得 (name)
        title_prop = properties.get("name", {})
        if title_prop and title_prop.get("title"):
            title_text = [text.get("plain_text", "") for text in title_prop.get("title", [])]
            article["title"] = "".join(title_text)

        # 作成日時を取得 (Created time)
        created_time_prop = properties.get("Created time", {})
        if created_time_prop:
            article["date"] = created_time_prop.get("created_time", "")

        # タグを取得 (tag - select型)
        tag_prop = properties.get("tag", {})
        if tag_prop and tag_prop.get("select"):
            article["category"] = tag_prop.get("select", {}).get("name", "")

        # ピックアップタイプを取得 (pickup_type - select型)
        pickup_type_prop = properties.get("pickup_type", {})
        if pickup_type_prop and pickup_type_prop.get("select"):
            pickup_type = pickup_type_prop.get("select", {}).get("name", "")
            if pickup_type:
                article["pickup_type"] = pickup_type

        # 要約コンテンツを取得 (summary - rich_text型)
        summary_prop = properties.get("summary", {})
        if summary_prop and summary_prop.get("rich_text"):
            summary_text = [text.get("plain_text", "") for text in summary_prop.get("rich_text", [])]
            article["content"] = "".join(summary_text)

        # abstract情報も取得 (abstract - rich_text型)
        abstract_prop = properties.get("abstract", {})
        if abstract_prop and abstract_prop.get("rich_text"):
            abstract_text = [text.get("plain_text", "") for text in abstract_prop.get("rich_text", [])]
            # contentがすでにある場合は追加、ない場合は新規設定
            if "content" in article and article["content"]:
                article["content"] += " " + "".join(abstract_text)
            else:
                article["content"] = "".join(abstract_text)


        # URLを取得 (URL - url型)
        url_prop = properties.get("URL", {})
        if url_prop:
            article["url"] = url_prop.get("url", "")

        articles.append(article)

    df = pd.DataFrame(articles)

    return df

def setup_japanese_fonts():
    """日本語フォントをインストールして設定する"""
    try:

        # フォントファイルのパスを確認
        font_path = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'
        if not os.path.exists(font_path):
            font_path = '/usr/share/fonts/truetype/ipafont-gothic/ipag.ttf'

        # フォント設定
        font_prop = fm.FontProperties(fname=font_path)
        matplotlib.rcParams['font.family'] = font_prop.get_name()

        # フォントキャッシュの再構築
        fm._rebuild()

        print("日本語フォントの設定が完了しました")
        return font_prop
    except Exception as e:
        print(f"日本語フォント設定中にエラーが発生しました: {e}")
        return None

def preprocess_text(text):
    """テキストの前処理を行う"""
    # 小文字化
    text = text.lower()
    # 特殊文字や数字を削除
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text

def tokenize_japanese_simple(text):
    """日本語テキストをシンプルに単語に分割する"""
    if not isinstance(text, str) or not text:
        return []

    try:
        # 単語分割のみを行う
        tagger = MeCab.Tagger("-Owakati")
        words = tagger.parse(text).split()
        return words
    except Exception as e:
        print(f"形態素解析中にエラーが発生しました: {e}")
        # 簡易的な分割
        return text.split()

def is_likely_noun(word):
    """単語が名詞である可能性が高いかをヒューリスティックに判定"""
    # 単語の長さでフィルタリング（短すぎる単語は除外）
    if len(word) <= 1:
        return False

    # 助詞、助動詞の一般的なリスト
    particles = ['が', 'の', 'を', 'に', 'へ', 'と', 'から', 'より', 'で', 'や', 'は', 'も', 'に', 'て', 'で', 'ば', 'なら']
    if word in particles:
        return False

    # 動詞の終止形などの典型的なパターン
    verb_endings = ['する', 'れる', 'られる', 'せる', 'させる', 'なる', 'ある', 'いる']
    for ending in verb_endings:
        if word.endswith(ending):
            return False

    # カタカナ語は技術用語である可能性が高い
    if any(c in 'アイウエオカキクケコサシスセソタチツテトナニヌネノハヒフヘホマミムメモヤユヨラリルレロワヲンガギグゲゴザジズゼゾダヂヅデドバビブベボパピプペポ' for c in word):
        return True

    # 英数字を含む単語は技術用語である可能性が高い
    if any(c.isascii() for c in word):
        return True

    # その他の単語は3文字以上であれば名詞の可能性が高い
    return len(word) >= 3

def create_word_frequency(df, column='content', english_only=False):
    """テキストから技術用語の単語頻度を計算する"""
    stopwords = get_stopwords()
    all_words = []

    for text in df[column].fillna(""):
        if not isinstance(text, str) or not text:
            continue

        # 単語分割
        tokens = tokenize_japanese_simple(text)

        if english_only:
            # 英語の技術用語のみをフィルタリング
            filtered_tokens = [word for word in tokens if is_english_tech_term(word)]
        else:
            # 名詞である可能性が高い単語のみをフィルタリング
            filtered_tokens = [word for word in tokens if is_likely_noun(word) and word not in stopwords]

        all_words.extend(filtered_tokens)

    # 単語の出現回数をカウント
    word_freq = Counter(all_words)

    # 頻度が低すぎる単語は除外（ノイズ除去）
    word_freq = Counter({word: count for word, count in word_freq.items() if count > 1})

    print(f"抽出された{'英語' if english_only else ''}技術用語: {', '.join(list(word_freq.keys())[:20])}")
    print(f"単語頻度データを生成しました（{len(word_freq)}個の単語）")

    return word_freq

def get_stopwords():
    """拡張版日本語のストップワードを取得"""
    # 基本的なストップワード
    ja_stops = set(['の', 'に', 'は', 'を', 'た', 'が', 'で', 'て', 'と', 'し', 'れ', 'さ', 'ある', 'いる', 'する', 'ない',
                     'できる', 'なる', 'もの', 'こと', 'これ', 'それ', 'あれ', 'この', 'その', 'あの', 'ます', 'です',
                     'だ', 'した', 'して', 'しない', 'なった', 'なって', 'なる', 'なり', 'など', 'とき', 'ところ'])

    # 追加の一般的な助詞・助動詞
    ja_stops.update(['も', 'や', 'へ', 'から', 'より', 'によって', 'において', 'について', 'として',
                    'ため', 'ための', 'ように', 'ような', 'らしい', 'れる', 'られる'])

    # 一文字語
    ja_stops.update(['ま', 'い', 'す', 'あ', 'お', 'く', 'き', 'け', 'さ', 'し', 'せ', 'そ', 'た', 'ち', 'つ', 'て', 'と',
                    'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ひ', 'ふ', 'へ', 'ほ', 'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ',
                    'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん'])

    # 分析対象の記事内容に合わせて追加の分野特有語・助詞・単語
    tech_common_words = [
        'あり', 'あれ', 'いう', 'いく', 'いっ', 'おく', 'おり', 'から', 'くる', 'くれ', 'この', 'これ',
        'させる', 'さらに', 'さまざま', 'しまう', 'しまった', 'している', 'してき', 'してきた',
        'すべて', 'その', 'それ', 'たくさん', 'ため', 'とき', 'ところ', 'とは', 'との', 'なっ',
        'なった', 'なって', 'なり', 'なる', 'など', 'について', 'による', 'または', 'また', 'まで',
        'まし', 'もの', 'よう', 'より', 'られ', 'られる', 'れた', 'れて', 'れる', 'わかる',
        'でき', 'できる', 'でも', 'という', 'への', 'ほとんど', 'もの', 'やすい', 'よる',
        'いる', 'ある', 'する', 'せる', 'こと', 'いくつか', 'どの', 'どう', 'どんな',

        # 技術単語
        'ソース', 'モード', '性能', '画像', '投稿', '活用', '可能', '利用', '提供', '開発',
        '導入', '機能', '発表', '公開', '対応', '実現', '向け', '向上', '必要', '課題', '問題',
        '方法', '取り組み', '研究', '技術', '企業', '情報', '最新', '新た', '新しい', '重要',
        '多く', '生成', '予定', '自動', '簡単', '強化', '画像', '使用', '目的', '特徴', '改善'
    ]

    ja_stops.update(tech_common_words)

    return ja_stops


def generate_wordcloud(word_freq, title, output_file, max_words=100):
    """ワードクラウドを生成して保存する"""
    if not word_freq:
        print("単語頻度データが空のため、ワードクラウドを生成できません")
        return

    # カラーマップの設定
    colors = ["#66c2a5", "#fc8d62", "#8da0cb", "#e78ac3", "#a6d854"]
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=100)

    # 日本語フォントの設定（Colab用）
    try:
        # 日本語フォントをインストール
        import subprocess
        subprocess.run(['apt-get', 'update'], check=True)
        subprocess.run(['apt-get', 'install', '-y', 'fonts-noto-cjk'], check=True)

        # フォントパスを直接指定
        font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'

        wordcloud = WordCloud(
            font_path=font_path,
            width=800,
            height=400,
            background_color='white',
            colormap=cmap,
            max_words=max_words,
            collocations=False
        ).generate_from_frequencies(word_freq)
    except Exception as e:
        print(f"日本語フォント設定エラー: {e}")
        print("フォントなしでワードクラウドを生成します")
        # フォントパスなしでワードクラウドを生成
        wordcloud = WordCloud(
            width=800,
            height=400,
            background_color='white',
            colormap=cmap,
            max_words=100,
            collocations=False,
            font_path=None
        ).generate_from_frequencies(word_freq)

    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=15)
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"ワードクラウドを保存しました: {output_file}")

def create_topic_model_sklearn(df, column='content', num_topics=5, english_only=False):
    """トピックモデリング（LDA）をscikit-learnで実行する - 英語フィルタリング対応版"""
    if column not in df.columns or df[column].isna().all():
        print(f"カラム '{column}' が存在しないか、すべての値が空です")
        return {}, None, None, None

    try:
        # テキストを前処理
        stopwords = get_stopwords()
        processed_docs = []

        for text in df[column].fillna(""):
            if not isinstance(text, str) or not text:
                processed_docs.append("")
                continue

            # 単語分割
            tokens = tokenize_japanese_simple(text)

            if english_only:
                # 英語の技術用語のみをフィルタリング
                filtered_tokens = [word for word in tokens if is_english_tech_term(word)]
            else:
                # 名詞である可能性が高い単語のみをフィルタリング
                filtered_tokens = [word for word in tokens if is_likely_noun(word) and word not in stopwords]

            processed_docs.append(" ".join(filtered_tokens))

        # 空のドキュメントをフィルタリング
        processed_docs = [doc for doc in processed_docs if doc]

        if not processed_docs:
            print("有効なドキュメントがありません")
            return {}, None, None, None

        # トピック数の調整（データが少ない場合）
        if len(processed_docs) < num_topics:
            num_topics = max(2, min(5, len(processed_docs) // 2))
            print(f"データ数に合わせてトピック数を {num_topics} に調整しました")

        # CountVectorizerでBoW (Bag of Words) を作成
        vectorizer = CountVectorizer(max_features=1000)
        X = vectorizer.fit_transform(processed_docs)

        if X.shape[1] == 0:
            print("抽出された特徴量がありません")
            return {}, None, None, None

        # LDAモデルを訓練
        lda = LatentDirichletAllocation(
            n_components=num_topics,
            random_state=42,
            max_iter=10,
            learning_method='online'
        )
        lda.fit(X)

        # 特徴量（単語）の名前を取得
        feature_names = vectorizer.get_feature_names_out()

        # トピックごとの上位単語を抽出
        topics = {}
        for idx, topic in enumerate(lda.components_):
            top_words_idx = topic.argsort()[:-21:-1]  # 上位20語のインデックスを取得（より多くの単語を取得）
            top_words = [feature_names[i] for i in top_words_idx]
            topics[f"Topic {idx+1}"] = top_words

        # 各ドキュメントのトピック分布を取得
        doc_topic_dist = lda.transform(X)

        return topics, doc_topic_dist, vectorizer, lda
    except Exception as e:
        print(f"トピックモデリング中にエラーが発生しました: {e}")
        return {}, None, None, None
def plot_topic_distribution_sklearn(doc_topic_dist, topics, num_topics, output_file):
    """記事のトピック分布を可視化する（scikit-learn版） - 改良版"""
    if doc_topic_dist is None or doc_topic_dist.size == 0 or not topics:
        print("トピック分布データがないため、グラフを生成できません")
        return

    try:
        # 各ドキュメントの主要トピックを取得
        dominant_topics = np.argmax(doc_topic_dist, axis=1)

        # トピック分布のカウント
        topic_counts = Counter(dominant_topics)

        # トピックの表示名を作成（上位3単語を含める）
        topic_labels = []
        for i in range(num_topics):
            if f"Topic {i+1}" in topics and topics[f"Topic {i+1}"]:
                top_words = topics[f"Topic {i+1}"][:3]  # 上位3単語
                label = f"Topic {i+1}\n({', '.join(top_words)})"
            else:
                label = f"Topic {i+1}"
            topic_labels.append(label)

        counts = [topic_counts.get(i, 0) for i in range(num_topics)]

        # 棒グラフの作成
        plt.figure(figsize=(12, 7))
        ax = sns.barplot(x=topic_labels, y=counts)
        plt.title("記事のトピック分布", fontsize=16)
        plt.xlabel("トピック（上位3キーワード）", fontsize=14)
        plt.ylabel("記事数", fontsize=14)
        plt.xticks(rotation=15, ha='right')

        # バーの上に数値を表示
        for i, v in enumerate(counts):
            ax.text(i, v + 0.5, str(v), ha='center')

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"トピック分布図を保存しました: {output_file}")
    except Exception as e:
        print(f"トピック分布図の作成中にエラーが発生しました: {e}")

def plot_category_distribution(df, output_file):
    """タグ（カテゴリ）の分布を可視化する """
    if 'category' not in df.columns or df['category'].isna().all():
        print("カテゴリデータがないため、カテゴリ分布図を生成できません")
        return

    try:
        # 空の値を除外
        category_counts = df['category'].dropna().value_counts()

        if len(category_counts) == 0:
            print("有効なカテゴリがないため、カテゴリ分布図を生成できません")
            return

        # 多すぎる場合は上位20個に制限
        if len(category_counts) > 20:
            category_counts = category_counts.head(20)
            print(f"カテゴリが多いため、上位20個のみを表示します")

        plt.figure(figsize=(20, 8))
        ax = sns.barplot(x=category_counts.index, y=category_counts.values)
        plt.title("記事のタグ分布 (上位20件)", fontsize=16)
        plt.xlabel("タグ", fontsize=14)
        plt.ylabel("記事数", fontsize=14)

        # x軸ラベルの回転とレイアウト調整
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.subplots_adjust(bottom=0.3)  # 下部の余白を増やす

        # バーの上に数値を表示
        for i, v in enumerate(category_counts.values):
            ax.text(i, v + 0.5, str(v), ha='center')

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"カテゴリ分布図を保存しました: {output_file}")
    except Exception as e:
        print(f"カテゴリ分布図の作成中にエラーが発生しました: {e}")

def plot_time_series(df, output_file):
    """時系列での記事数の推移を可視化する"""
    if 'date' not in df.columns:
        return

    # 日付をdatetime型に変換
    df['date'] = pd.to_datetime(df['date'])

    # 日付ごとに記事数を集計
    daily_counts = df.groupby(df['date'].dt.date).size()

    # 折れ線グラフの作成
    plt.figure(figsize=(12, 6))
    plt.plot(daily_counts.index, daily_counts.values, marker='o', linestyle='-', color='#3498db')
    plt.title("日別記事数の推移", fontsize=15)
    plt.xlabel("日付", fontsize=12)
    plt.ylabel("記事数", fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

def analyze_category_co_occurrence(df, output_file):
    """カテゴリの共起関係を分析して可視化する"""
    if 'category' not in df.columns or df['category'].isna().all():
        print("categoryカラムがないか空のため、カテゴリネットワーク図を生成できません")
        return False

    try:


        # カテゴリの出現頻度をカウント
        category_counts = df['category'].value_counts().to_dict()

        # 共起関係を調べるため、同じ日に出現したカテゴリのペアを作成
        df['date'] = pd.to_datetime(df['date']).dt.date
        category_pairs = []

        for date, group in df.groupby('date'):
            categories = group['category'].dropna().unique()
            for i, cat1 in enumerate(categories):
                for cat2 in categories[i+1:]:
                    pair = tuple(sorted([cat1, cat2]))
                    category_pairs.append(pair)

        category_co_occurrence = Counter(category_pairs)

        # 共起頻度が多い上位のカテゴリペアを取得
        top_pairs = category_co_occurrence.most_common(15)
        if not top_pairs:
            print("カテゴリの共起関係がないため、カテゴリネットワーク図を生成できません")
            return False

        print(f"カテゴリの共起関係: {top_pairs[:5]}")

        # 日本語フォントのパス
        font_path = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'
        if not os.path.exists(font_path):
            font_path = '/usr/share/fonts/truetype/ipafont-gothic/ipag.ttf'

        # ネットワーク図のためのノードとエッジを作成
        import networkx as nx
        G = nx.Graph()

        # すべてのカテゴリを抽出
        all_categories = set()
        for pair, count in top_pairs:
            cat1, cat2 = pair
            all_categories.add(cat1)
            all_categories.add(cat2)

        # ノードを追加（出現頻度に基づいてサイズを調整）
        for category in all_categories:
            G.add_node(category, size=category_counts.get(category, 1))

        # エッジを追加
        for pair, count in top_pairs:
            cat1, cat2 = pair
            G.add_edge(cat1, cat2, weight=count)

        # ネットワーク図の作成
        plt.figure(figsize=(14, 10))
        pos = nx.spring_layout(G, seed=42)

        # ノードのサイズを調整
        node_size = [100 + 20 * G.nodes[node]['size'] for node in G.nodes()]

        # エッジの太さを調整
        edge_width = [0.5 + G[u][v]['weight'] for u, v in G.edges()]

          # ネットワーク図の描画
        nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='#3498db', alpha=0.7)
        nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5, edge_color='#7f8c8d')

        # 日本語フォントプロパティの設定
        font_prop = FontProperties(fname=font_path)

        # ラベルを手動で描画
        for node, (x, y) in pos.items():
            plt.text(x, y, node, fontproperties=font_prop,
                   size=12, ha='center', va='center',
                   bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.3'))


        plt.title("カテゴリの共起ネットワーク", fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"カテゴリネットワーク図を保存しました: {output_file}")
        return True
    except Exception as e:
        print(f"カテゴリネットワーク図の作成中にエラーが発生しました: {e}")
        return False

def is_english_tech_term(word):
    """英語の技術用語かどうかを判定"""
    # 英数字のみで構成されている
    if word.isascii() and not word.isdigit():
        # 一般的な英語のストップワードリスト
        eng_stopwords = {
            'the', 'and', 'to', 'of', 'a', 'in', 'for', 'is', 'on', 'that', 'by',
            'this', 'with', 'it', 'as', 'are', 'was', 'be', 'at', 'from', 'has',
            'have', 'had', 'an', 'but', 'or', 'if', 'they', 'their', 'what', 'which',
            'when', 'who', 'how', 'where', 'why', 'not', 'all', 'any', 'can', 'will'
        }

        additional_stopwords = {
            'is', 'do', 'get', 'got', 'vs', 'mini', 'deep', 'lab', 'see',
            'go', 'next', 'line', 'mac', 'fp', 'ceo', 'dx',
            'pro', 'ui', 'pc', 'de', 'ai', 'openai', 'google', 'apple', 'amazon','microsoft'
        }

        # 両方のストップワードセットを結合
        all_stopwords = eng_stopwords.union(additional_stopwords)

        # 小文字に変換して判定
        lower_word = word.lower()

        # 長さが2以上で、ストップワードではない場合
        return len(word) >= 2 and lower_word not in all_stopwords
    return False

def generate_weekly_report(start_date=None, end_date=None, output_dir="reports",english_only=False):
    """週間トレンドレポートを生成する"""
    # 日付が指定されていない場合は先週を対象にする
    if not start_date or not end_date:
        today = datetime.now()
        end_date = today.strftime("%Y-%m-%d")
        start_date = (today - timedelta(days=7)).strftime("%Y-%m-%d")

    # 日付をYYYYMMDD_YYYYMMDD形式に変換してファイル名用の文字列を生成
    start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")
    date_range_str = f"{start_date_obj.strftime('%Y%m%d')}_{end_date_obj.strftime('%Y%m%d')}"

    print(f"期間: {start_date} から {end_date} のレポートを生成します")

    # 出力ディレクトリに日付範囲を含める
    output_dir = f"{output_dir}/{date_range_str}"
    os.makedirs(output_dir, exist_ok=True)

    # データを取得
    results = query_database(start_date, end_date)
    if not results:
        print("指定された期間のデータが見つかりませんでした")
        return

    # データをDataFrameに変換
    df = extract_properties(results)
    print(f"{len(df)}件の記事が見つかりました")

    # 基本的な統計情報を表示
    if 'category' in df.columns and not df['category'].isna().all():
        print("タグ別の記事数:")
        print(df['category'].value_counts())

    if 'pickup_type' in df.columns and not df['pickup_type'].isna().all():
        print("\nピックアップタイプ別の記事数:")
        print(df['pickup_type'].value_counts())

    # ワードクラウドの生成
    word_freq = None
    if 'content' in df.columns and not df['content'].isna().all():
        word_freq = create_word_frequency(df, english_only=english_only)
        if word_freq:
            generate_wordcloud(
                word_freq,
                f"{'英語' if english_only else ''}キーワード頻度 ({start_date} ~ {end_date})",
                f"{output_dir}/wordcloud.png",
                max_words=100
            )
            print("ワードクラウドを生成しました")
        else:
            print("警告: 単語頻度データが空のため、ワードクラウドを生成できませんでした")

    # トピックモデリング
    topics = {}
    if 'content' in df.columns and not df['content'].isna().all() and len(df) >= 5:
        topics, doc_topic_dist, vectorizer, lda = create_topic_model_sklearn(df, english_only=english_only)
        try:
          if topics and doc_topic_dist is not None and lda is not None:
              print("\nトピックモデル:")
              for topic_name, words in topics.items():
                  print(f"{topic_name}: {', '.join(words[:7])}")

              plot_topic_distribution_sklearn(doc_topic_dist, topics, lda.n_components, f"{output_dir}/topic_distribution.png")
        except Exception as e:
            print(f"トピックモデル作成中にエラーが発生しました: {e}")
            topics = {}
    else:
        topics = {}
        if 'content' not in df.columns or df['content'].isna().all():
            print("コンテンツデータがないため、トピックモデリングをスキップします")

    # カテゴリ（タグ）分布の可視化
    plot_category_distribution(df, f"{output_dir}/category_distribution.png")


    # 時系列分析
    if 'date' in df.columns and not df['date'].isna().all():
        try:
            plot_time_series(df, f"{output_dir}/time_series.png")
            print("時系列グラフを生成しました")
        except Exception as e:
            print(f"時系列グラフ作成中にエラーが発生しました: {e}")


    # カテゴリネットワーク図の生成
    category_network_generated = analyze_category_co_occurrence(df, f"{output_dir}/category_network.png")
    if category_network_generated:
        print("カテゴリネットワーク図を生成しました")
    else:
        print("カテゴリネットワーク図の生成をスキップしました")

    # レポート結果をJSONとして保存
    category_data = {}
    if 'category' in df.columns and not df['category'].isna().all():
        category_data = df['category'].value_counts().to_dict()

    pickup_data = {}
    if 'pickup_type' in df.columns and not df['pickup_type'].isna().all():
        pickup_data = df['pickup_type'].value_counts().to_dict()

    flag_data = {}
    if 'flag' in df.columns and not df['flag'].isna().all():
        flag_data = df['flag'].value_counts().to_dict()

    keyword_data = {}
    if word_freq:
        keyword_data = dict(word_freq.most_common(50))

    topic_data = {}
    if topics:
        for topic_name, words in topics.items():
            topic_data[topic_name] = words[:10]

    report_data = {
        "period": {
            "start_date": start_date,
            "end_date": end_date
        },
        "article_count": len(df),
        "categories": category_data,
        "pickup_types": pickup_data,
        "flags": flag_data,
        "top_keywords": keyword_data,
        "topics": topic_data
    }

    with open(f"{output_dir}/report_data.json", "w", encoding="utf-8") as f:
        json.dump(report_data, f, ensure_ascii=False, indent=2)

    print(f"レポートデータをJSONとして保存しました: {output_dir}/report_data.json")

    return {
        "df": df,
        "report_data": report_data,
        "output_dir": output_dir
    }


In [ ]:
start_date = "2025-03-21"
end_date = "2025-04-17"

output_dir = f"/content/drive/MyDrive/weekly_report"
report = generate_weekly_report(start_date, end_date, output_dir, english_only=True)